In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

from dotenv import load_dotenv
import os
load_dotenv()
key = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model="gpt-4.1-nano", openai_api_key=key)


In [2]:
from langchain_core.tools import tool
from datetime import datetime
import pytz

@tool
def get_current_time(timezone:str,location:str)->str:
    # (이렇게 앞쪽에 설명안써주면 에러뜸)
    """현재시간 반환 

    Args:
        timezone(str): 타임존 (예:'asia/seoul') 실제 존재하는 타임존이어야 함.
        location(str): 지역명. 타임존이 모든 지명에 대응되지 않기 떄문에 이후 llm 답변 생성에 사용됨
    """
    tz = pytz.timezone(timezone)
    now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
    location_and_local_time = f"{timezone} ({location}) 현재시각 {now}"
    print(location_and_local_time)
    return location_and_local_time

tools = [get_current_time,]
tool_dict = {"get_current_time":get_current_time,}

llm_with_tools = llm.bind_tools(tools)

messages = [
    SystemMessage("너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
    HumanMessage("부산은 지금 몇시야?")
]

In [3]:
response = llm_with_tools.invoke(messages)
messages.append(response)

In [4]:
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]]
    print(tool_call)
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': 'Busan'}, 'id': 'call_oBmZefmCsXbvLm3mwJgz5gFU', 'type': 'tool_call'}
Asia/Seoul (Busan) 현재시각 2025-08-22 11:07:41


In [5]:
tool_msg

ToolMessage(content='Asia/Seoul (Busan) 현재시각 2025-08-22 11:07:41', name='get_current_time', tool_call_id='call_oBmZefmCsXbvLm3mwJgz5gFU')

In [6]:
messages

[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_oBmZefmCsXbvLm3mwJgz5gFU', 'function': {'arguments': '{"timezone": "Asia/Seoul", "location": "Busan"}', 'name': 'get_current_time'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 129, 'total_tokens': 168, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_c4c155951e', 'id': 'chatcmpl-C7BTpzIWh3Yb4IbflnOKWCXBsfIcg', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--089001eb-5cff-4488-bd79-8c4645ace88e-0', t

In [8]:
finalMsg = llm_with_tools.invoke(messages)

In [9]:
print(finalMsg)

content='현재 부산의 시간은 2025년 8월 22일 11시 07분입니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 185, 'total_tokens': 209, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_c4c155951e', 'id': 'chatcmpl-C7BU12ZpqhidMl5Z1jQVIWgopAfOG', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--b5264a1d-4cad-4947-b25c-32340f55b838-0' usage_metadata={'input_tokens': 185, 'output_tokens': 24, 'total_tokens': 209, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
